# All Saxophones: SVM

* model = SVM
* target = TENOR and ALTO and SOPRANO and BARITONE (ie. ALL SAXES)

Use SVM model to classify whether or not a saxophone is playing in a sample.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import BaggingClassifier
from scipy.stats import uniform

from pickles_to_pandas import pickles_to_pandas


In [2]:
df = pickles_to_pandas('./data/5s/labeled/features_r02')

In [3]:
NUM_LABEL_COLS = 13
NUM_COMPONENTS = 30

In [4]:
df_filtered = df[df['excl'] == '0']  # exclude records we want to exclude
df_filtered.shape

(5411, 10790)

In [5]:
# exclude records we want to exclude
df_filtered = df_filtered[df_filtered['clrt'] == '0']
df_filtered = df_filtered[df_filtered['othr'] == '0']
df_filtered = df_filtered[df_filtered['trmp'] == '0']
df_filtered = df_filtered[df_filtered['trmb'] == '0']
df_filtered = df_filtered[df_filtered['otrb'] == '0']

num_x_cols = df_filtered.shape[1] - NUM_LABEL_COLS - 1  
# ^ last bit to adjust for zero indexing
data = df_filtered.iloc[:, 1:num_x_cols].to_numpy() 
# ^ These are the features
scaler = StandardScaler()
scaler.fit(data)
data = scaler.transform(data)

# Create target column
combined = df_filtered[['tenr']].to_numpy() + df_filtered[['alto']].to_numpy() + df_filtered[['tora']].to_numpy() + \
            df_filtered[['sop']].to_numpy() + df_filtered[['bari']].to_numpy()
combined = combined.astype('int')
combined[combined > 0] = 1
df_filtered['saxophones'] = combined

target = df_filtered[['saxophones']].to_numpy().ravel()  # << This is the label

print(data.shape)
print(target.shape)

(4365, 10775)
(4365,)


In [6]:
pca = PCA(n_components=NUM_COMPONENTS)

In [7]:
pca.fit(data)

PCA(n_components=30)

In [8]:
# print(pca.explained_variance_ratio_)

In [9]:
# print(pca.singular_values_)

In [10]:
d = pca.transform(data)

In [11]:
# d

In [12]:
x_train, x_test, y_train, y_test = \
     train_test_split(d, target, test_size=0.20, random_state=0)

## Test plain vanilla SVM

In [13]:
clf = SVC()
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))

0.8694158075601375


In [14]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,316,56
1,58,443


In [15]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.85      0.85       372
           1       0.89      0.88      0.89       501

    accuracy                           0.87       873
   macro avg       0.87      0.87      0.87       873
weighted avg       0.87      0.87      0.87       873



## Determine good value for C using random search

In [16]:
model_params = {
    'C': uniform(0.1, 3.0)
}

# RandomizedSearchCV(estimator, param_distributions, *, n_iter=10, 
#                    scoring=None, n_jobs=None, iid='deprecated', 
#                    refit=True, cv=None, verbose=0, 
#                    pre_dispatch='2*n_jobs', random_state=None, 
#                    error_score=nan, return_train_score=False)
svm_model = SVC()
clf = RandomizedSearchCV(svm_model, model_params, n_iter=10, 
                         cv=5)
model = clf.fit(x_train, y_train)
params = model.best_estimator_.get_params()

In [17]:
clf = SVC(C=params['C'])
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))


0.8762886597938144


In [18]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,323,49
1,59,442


In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.87      0.86       372
           1       0.90      0.88      0.89       501

    accuracy                           0.88       873
   macro avg       0.87      0.88      0.87       873
weighted avg       0.88      0.88      0.88       873



## Test ensemble approach with bagging classifier

In [20]:
# BaggingClassifier(base_estimator=None, n_estimators=10, *, 
#                   max_samples=1.0, max_features=1.0, 
#                   bootstrap=True, bootstrap_features=False, 
#                   oob_score=False, warm_start=False, 
#                   n_jobs=None, random_state=None, verbose=0) 
clf = BaggingClassifier(base_estimator=SVC(C=params['C']),
                        n_estimators=400, 
                        max_features=NUM_COMPONENTS - 4,
                        random_state=42)
model = clf.fit(x_train, y_train)

In [21]:
y_pred = clf.predict(x_test)
print(clf.score(x_test, y_test))

0.8797250859106529


In [22]:
c_matrix = confusion_matrix(y_test, y_pred)
idx = cols = [0, 1]
pd.DataFrame(c_matrix, index=idx, columns=cols)

,0,1
0,321,51
1,54,447


In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.86      0.86       372
           1       0.90      0.89      0.89       501

    accuracy                           0.88       873
   macro avg       0.88      0.88      0.88       873
weighted avg       0.88      0.88      0.88       873

